In [37]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier

In [38]:
import requests
import pandas as pd
from bs4 import BeautifulSoup


Ajith Movies

In [39]:
import pandas as pd
import requests

url = "https://en.wikipedia.org/wiki/Ajith_Kumar_filmography"

headers = {"User-Agent": "Mozilla/5.0"}
html = requests.get(url, headers=headers).text

# Read all tables from the page
tables = pd.read_html(html)

print("Total tables found:", len(tables))


Total tables found: 2


/tmp/ipython-input-3932053093.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


In [40]:
for i, table in enumerate(tables):
    print(i, table.shape)


0 (1, 2)
1 (63, 5)


In [41]:
import pandas as pd
import requests

url = "https://en.wikipedia.org/wiki/Ajith_Kumar_filmography"
headers = {"User-Agent": "Mozilla/5.0"}

html = requests.get(url, headers=headers).text

# Read tables
tables = pd.read_html(html)

# Find the correct film table (largest one)
film_table = max(tables, key=lambda x: len(x))

# Rename columns cleanly
film_table.columns = ["Year", "Film", "Role", "Notes", "Ref"]

# ✅ CRITICAL STEP: fill missing years caused by rowspan
film_table["Year"] = film_table["Year"].ffill()

# Remove citation brackets from film names
film_table["Film"] = film_table["Film"].str.replace(r"\[.*?\]", "", regex=True)

# Save properly
film_table.to_csv("ajith.csv", index=False)

print("Saved ajith.csv with", len(film_table), "movies")

Saved ajith.csv with 63 movies


/tmp/ipython-input-1352439078.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


In [42]:
df = pd.read_csv('ajith.csv')
df

,Year,Film,Role,Notes,Ref
0,1990,En Veedu En Kanavar,School boy,Child artist,[28]
1,1993,Amaravathi,Arjun,Debut as lead actor,[29][30]
2,1993,Prema Pusthakam,Sreekar,Telugu film,[5][31]
3,1994,Paasamalargal,Kumar,NaN,[5]
4,1994,Pavithra,Ashok,NaN,[32]
...,...,...,...,...,...
58,2019,Nerkonda Paarvai,Bharath Subramaniam,NaN,[84]
59,2022,Valimai,Arjun Kumar,NaN,[85]
60,2023,Thunivu,Dark Devil,NaN,[86]
61,2025,Vidaamuyarchi,Arjun,NaN,[87]


Vijay

In [43]:
import pandas as pd
import requests

url = "https://en.wikipedia.org/wiki/Vijay_filmography"
headers = {"User-Agent": "Mozilla/5.0"}

html = requests.get(url, headers=headers).text

# Read all tables
tables = pd.read_html(html)

# 🔍 Identify the correct table: Vijay filmography is the largest one
vijay_table = max(tables, key=lambda x: len(x))

print("Rows found:", len(vijay_table))

# Rename columns (Wikipedia sometimes changes wording)
vijay_table.columns = ["Year", "Film", "Role", "Notes", "Ref"]

# ✅ Handle rowspan years
vijay_table["Year"] = vijay_table["Year"].ffill()

# Clean film names (remove citation brackets)
vijay_table["Film"] = vijay_table["Film"].str.replace(r"\[.*?\]", "", regex=True)

# Optional: drop Ref column (not useful for ML)
vijay_table = vijay_table.drop(columns=["Ref"])

# Save
vijay_table.to_csv("vijay.csv", index=False)

print("Saved vijay.csv with", len(vijay_table), "movies")


Rows found: 77
Saved vijay.csv with 77 movies


/tmp/ipython-input-3970069057.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


In [44]:
df = pd.read_csv('vijay.csv')
df

,Year,Film,Role,Notes
0,1984,Vetri,Vijay,Child Actor
1,1984,Kudumbam,Narada,Child Actor
2,1985,Naan Sigappu Manithan,Unnamed,Child Actor
3,1986,Vasantha Raagam,Vijay,Child Actor
4,1987,Sattam Oru Vilayattu,Raja,Child Actor
...,...,...,...,...
72,2022,Beast,Veera Raghavan,NaN
73,2023,Varisu,Vijay Rajendran,NaN
74,2023,Leo,Parthiban/Leo Das,NaN
75,2024,The Greatest Of All Time,"M.S. Gandhi, Jeevan Gandhi (Sanjay Menon) [b]",NaN


In [50]:
import pandas as pd

basics = pd.read_csv(
    "name.basics.tsv",
    sep="\t",
    dtype=str,
    low_memory=False
)

ratings = pd.read_csv(
    "title.ratings.tsv",
    sep="\t",
    dtype=str
)

print(basics.shape)
print(ratings.shape)


(15058298, 6)
(1631032, 3)


In [51]:
movies = basics[
    (basics["titleType"] == "movie") &
    (basics["startYear"] != "\\N")
][["tconst", "primaryTitle", "startYear"]]

movies["startYear"] = movies["startYear"].astype(int)


KeyError: 'titleType'

In [ ]:
ratings["numVotes"] = ratings["numVotes"].astype(int)
ratings["averageRating"] = ratings["averageRating"].astype(float)

movies = movies.merge(ratings, on="tconst", how="left")

In [ ]:
vijay = pd.read_csv("vijay.csv")

vijay["Year"] = vijay["Year"].astype(int)


In [ ]:
vijay_imdb = vijay.merge(
    movies,
    left_on=["Film", "Year"],
    right_on=["primaryTitle", "startYear"],
    how="left"
)


In [ ]:
final_vijay = vijay_imdb[
    ["Film", "Year", "Role", "IMDb_Rating", "IMDb_Votes"]
].rename(columns={
    "averageRating": "IMDb_Rating",
    "numVotes": "IMDb_Votes"
})

final_vijay.to_csv("vijay_imdb.csv", index=False)

print("Saved vijay_imdb.csv with", len(final_vijay), "movies")


In [ ]:
final_vijay[final_vijay["IMDb_Rating"].notna()].head()
